In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from trajectory.segments import * 
from trajectory.planner import * 
from trajectory.plot import * 
from trajectory.trapmath import * 
import matplotlib.pyplot as plt
from IPython.display import display
import trajectory as tj
import pandas as pd
import numpy as np
from math import sqrt
from random import randint, random
from operator import attrgetter
from itertools import chain
pd.set_option('display.max_columns', None)

from copy import deepcopy

ag_ha = attrgetter(*'t v_0 v_c v_1 a_max'.split())
ag = attrgetter(*'x t v_0 v_c v_1 v_0_max v_1_max'.split())

def ch(v):
    for e in chain(*v):
        if e is not None:
            yield e
            


In [2]:
j = Joint(5_000, 50_000)
joints = [j]*3
x = 1000

sl = SegmentList(joints)   


s = sl.rmove([0,x,x])
s = sl.rmove([0,x/2,x])
s = sl.rmove([0,x/2,0])
s = sl.rmove([x,x,x])

print(sl)
sl.plot(); plt.show()
sl.dataframe

NameError: name 'min_time_parameters' is not defined

In [ ]:
pd.Series( index=['t', 'seg', 'axis', 'x', 'v_i', 'v_f', 'ss', 'del_t', 'v0m', 'v1m', 'calc_x', 'err'], dtype=np.float64)

In [ ]:
j = Joint(5_000, 50_000)
joints = [j]*3
x = 1000

sl = SegmentList(joints)   


moves = []
for i in range(40):
    move = [max(randint(-1000,2_000), 0) for _ in joints]
    moves.append(move)
    s = sl.rmove(move)
    
print(sl)
sl.plot(); plt.show()
sl.dataframe

In [ ]:
moves